<a href="https://colab.research.google.com/github/prashant-dot/Face-Recognition-System/blob/main/FRS_using_deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import cv2

In [2]:
def generate_dataset():
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
        
        if faces is ():
            return None
        for (x,y,w,h) in faces:
            cropped_face = img[y:y+h,x:x+w]
        return cropped_face
    
    cap = cv2.VideoCapture(0)
    img_id = 0
    if not cap.isOpened():
        raise IOError("Cannot open webcam")
    cv2.imshow('Input', frame)
    while True:
        ret, frame = cap.read()
        print(frame)
        if face_cropped(frame) is not None:
            img_id+=1
            face = cv2.resize(face_cropped(frame), (200,200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            file_name_path = "data/"+"Prashant."+str(img_id)+".jpg"
            #file_name_path = "Images for visualization/"+str(img_id)+'.jpg'
            cv2.imwrite(file_name_path, face)
            cv2.putText(face, str(img_id), (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2 )
            
            cv2.imshow("Cropped_Face", face)
            if cv2.waitKey(1)==13 or int(img_id)==20:
                break
                
    cap.release()
    cv2.destroyAllWindows()
    print("Collecting samples is completed !!!")
generate_dataset()


'def generate_dataset():\n    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")\n    def face_cropped(img):\n        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)\n        faces = face_classifier.detectMultiScale(gray, 1.3, 5)\n        \n        if faces is ():\n            return None\n        for (x,y,w,h) in faces:\n            cropped_face = img[y:y+h,x:x+w]\n        return cropped_face\n    \n    cap = cv2.VideoCapture(0)\n    img_id = 0\n    if not cap.isOpened():\n        raise IOError("Cannot open webcam")\n    cv2.imshow(\'Input\', frame)\n    while True:\n        ret, frame = cap.read()\n        print(frame)\n        if face_cropped(frame) is not None:\n            img_id+=1\n            face = cv2.resize(face_cropped(frame), (200,200))\n            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)\n            file_name_path = "data/"+"Prashant."+str(img_id)+".jpg"\n            #file_name_path = "Images for visualization/"+str(img_id)+\'.jpg\'\n      

In [ ]:
import numpy as np # pip install numpy

In [ ]:
def my_label(image_name):
    name = image_name.split('.')[-3] 
    if name=="Prashant":
        return np.array([1,0,0])
    elif name=="Mukesh":
        return np.array([0,1,0])
    elif name=="Peter":
        return np.array([0,0,1])

In [ ]:
import os
from random import shuffle
from tqdm import tqdm

In [ ]:
def my_data():
    data = []
    for img in tqdm(os.listdir("data")):
        path=os.path.join("data",img)
        img_data = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img_data = cv2.resize(img_data, (50,50))
        data.append([np.array(img_data), my_label(img)])
    shuffle(data)  
    return data

In [ ]:
data = my_data()

In [ ]:
train = data[:2400]  
test = data[2400:]
X_train = np.array([i[0] for i in train]).reshape(-1,50,50,1)
print(X_train.shape)
y_train = [i[1] for i in train]
X_test = np.array([i[0] for i in test]).reshape(-1,50,50,1)
print(X_test.shape)
y_test = [i[1] for i in test]

In [ ]:
import tensorflow as tf
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

In [ ]:
tf.reset_default_graph()
convnet = input_data(shape=[50,50,1])
convnet = conv_2d(convnet, 32, 5, activation='relu')
# 32 filters and stride=5 so that the filter will move 5 pixel or unit at a time
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)
convnet = fully_connected(convnet, 3, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate = 0.001, loss='categorical_crossentropy')
model = tflearn.DNN(convnet, tensorboard_verbose=1)
model.fit(X_train, y_train, n_epoch=12, validation_set=(X_test, y_test), show_metric = True, run_id="FRS" )


In [ ]:
def data_for_visualization():
    Vdata = []
    for img in tqdm(os.listdir("Images for visualization")):
        path = os.path.join("Images for visualization", img)
        img_num = img.split('.')[0] 
        img_data = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img_data = cv2.resize(img_data, (50,50))
        Vdata.append([np.array(img_data), img_num])
    shuffle(Vdata)
    return Vdata

In [ ]:
Vdata = data_for_visualization()

In [ ]:
import matplotlib.pyplot as plt   # pip install matplotlib

fig = plt.figure(figsize=(20,20))
for num, data in enumerate(Vdata[:20]):
    img_data = data[0]
    y = fig.add_subplot(5,5, num+1)
    image = img_data
    data = img_data.reshape(50,50,1)
    model_out = model.predict([data])[0]
    
    if np.argmax(model_out) == 0:
        my_label = 'Prashant'
    elif np.argmax(model_out) == 1:
        my_label = 'Manish'
    else:
        my_label = 'Paul'
        
    y.imshow(image, cmap='gray')
    plt.title(my_label)
    
    y.axes.get_xaxis().set_visible(False)
    y.axes.get_yaxis().set_visible(False)
plt.show()
